<a href="https://colab.research.google.com/github/primaa28/Facial-Emotion-Recognition-System./blob/master/Facial_Emotion_Recognition_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader


In [ ]:

df = pd.read_csv(r'/content/fer2013.csv')



df.head(35887)


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
24742,6,0 0 4 4 2 0 0 0 2 3 2 6 10 5 3 6 5 13 8 9 51 5...,Training
24743,3,8 28 41 55 61 59 55 82 84 80 88 135 117 105 12...,Training
24744,4,255 255 255 255 255 255 255 255 255 255 255 25...,Training
24745,6,75 76 80 82 88 93 102 112 124 130 137 144 155 ...,Training


In [ ]:
df.shape

(249, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43386 entries, 0 to 43385
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  43386 non-null  object
 1   pixels   43386 non-null  object
 2   Usage    43384 non-null  object
dtypes: object(3)
memory usage: 1017.0+ KB


In [ ]:
#converting the string values of pixels into array
def arr_conv (strg): 
    b=[]
    a=strg.split(" ")
    for elements in a:
        b.append(int (elements))
    
    return np.array(b)


In [ ]:
df["pixels_arr"]= df['pixels'].apply(arr_conv)


ValueError: ignored

In [ ]:
min(df["pixels_arr"][100])


In [ ]:
max(df["pixels_arr"][0])

In [ ]:
image,label = df["pixels_arr"][0],df["emotion"][0]
print('Shape:', image.shape, '\nLabel:', label)

In [ ]:
plt.imshow(df["pixels_arr"][106].reshape(48,48),cmap='gray');

In [ ]:
len(df['pixels_arr'][0])

In [ ]:
type(df['pixels_arr'][0])

In [ ]:
train_data_fer= torch.Tensor(df['pixels_arr']) ##important step

In [ ]:
train_data_fer[0]

In [ ]:
train_data_fer.shape

In [ ]:
train_data_fer= train_data_fer.reshape(35887,1,48,48) #reshaping to 1 color channel and 48X48 image

In [ ]:
train_data_fer.shape

In [ ]:
train_data_fer[0].shape # 1st image as a tensor

In [ ]:
train_data_fer[0:5].shape #first 5 images

In [ ]:
plt.imshow(train_data_fer[0].reshape(48,48),cmap='gray');

In [ ]:
class_names= ['Angry', 'Disgust','Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [ ]:
labels= list(df['emotion'][0:5])
labels

In [ ]:
np.set_printoptions(formatter=dict(int=lambda x: f'{x:5}')) # to widen the printed array
from torchvision.utils import make_grid
labels =list(df['emotion'][0:5])
print('Label:', np.array(labels))
print('Class: ', *np.array([class_names[i] for i in labels]))
images= train_data_fer[0:5]
im = make_grid(images, nrow=5)
plt.figure(figsize=(12,4))

plt.imshow(np.transpose(im, (1, 2, 0)));

In [ ]:
Y=torch.Tensor(df['emotion'])# target variable values


In [ ]:
Y

In [ ]:
Y.shape

In [ ]:
Y = Y.type(torch.LongTensor)


In [ ]:
Y.type()

In [ ]:
Y

In [ ]:
len(Y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(train_data_fer, Y,test_size=0.20, random_state=23)

In [ ]:
x_train[0].shape

In [ ]:
x_train.shape

In [ ]:
y_train.shape



In [ ]:
y_train

In [ ]:
train_dataset = TensorDataset(x_train, y_train)  #imp step

In [ ]:
test_dataset = TensorDataset(x_test, y_test)  #imp step

In [ ]:
train_dataset[0]

In [ ]:
train_dataset[0][0].shape

In [ ]:
test_dataset[0]

In [ ]:
test_dataset[0][0].shape

In [ ]:
type(train_dataset)

In [ ]:
train_dataset

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
"""
Initializing train/test dataloader object which splits the training dataset into small batches of 50 images each
and this can be used later in the CNN model  
"""
torch.manual_seed(101)
bat_sz=50
train_loader = DataLoader(train_dataset,batch_size=bat_sz,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=bat_sz,shuffle=False)

In [ ]:
len(train_dataset)

In [ ]:
len(test_dataset)

Then we have 7 emotions that we are predicting namely (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral), so we have 7 labels. We will be processing our inputs with a batch size of 50

In [ ]:
class CONVNN(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        self.conv1  = nn.Conv2d(1, 300 , kernel_size =5, stride=1,padding=2)#1 color channels as input and We will be using 5X5 filter/kernel and stride of 1 without any padding 
        self.norm1  = nn.BatchNorm2d(300)
        #self.drop1  = nn.Dropout2d(p=0.2)
        self.conv2  = nn.Conv2d(300, 150, 5 , 1,padding=2)
        self.norm2  = nn.BatchNorm2d(150)
        self.conv3  = nn.Conv2d(150, 75, 5 , 1,padding=2)
        self.norm3  = nn.BatchNorm2d(75)
        #self.drop2  = nn.Dropout2d(p=0.2)
        self.layer1 = nn.Linear(6*6*75,180) # we need to calculate the resulting number of matrices passing thru the conv layer and put that corressponding o/p as the linear layer input
        self.drop3  = nn.Dropout2d(p=0.1)
        self.layer2 = nn.Linear(180,84)
        self.layer3 = nn.Linear(84,7) #only 7 classes of Dogs and Cats
        
    def forward(self,x):
                         #self.drop1
        x= F.max_pool2d(             ( F.relu(    self.norm1(self.conv1(x)) )  )      ,2,2) #adding relu and max pooling with 2x2 kernel and stride of 2
        
        x= F.max_pool2d(             ( F.relu(    self.norm2(self.conv2(x)) )  )      ,2,2) #adding relu and max pooling the same line
        x= F.max_pool2d(             ( F.relu(    self.norm3(self.conv3(x)) )  )      ,2,2) #adding relu and max pooling with 2x2 kernel and stride of 2

        x= self.drop3(F.relu(      self.layer1(  x.view(-1,6*6*75)   ) ) )      #flattening by using View and no dropout has been added
        x= F.relu(self.layer2(x))
        x= F.log_softmax(  self.layer3(x), dim=1)          #multi class classificaiton
        
        return x

In [ ]:
torch.manual_seed(101)
model = CONVNN()#to instansiate the model as cuda use "model = CONVNN().to(device) " or "model.cuda()" so that whatever the tensors we are passing to the model will be saved in GPU and operations will be performed on that
model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
epochs = 8
train_loss= []
test_loss= []
train_acc=[]
test_acc = []

print(f'\nConvolutional Neural Network Model Metrics:\n')
print(f'\t This CNN model configuration has {epochs} epochs with each batch size of {bat_sz} images:\n')
for i in range(epochs):
    
    train_crt_pred = 0
    test_crt_pred = 0
    conf_mat= torch.FloatTensor([])
    
    for b,(x_train,y_train) in enumerate (train_loader):
        b += 1
        y_pred = model.forward(x_train)
        loss= criterion(y_pred,y_train.long() )
        
        buffer = torch.max(y_pred.data, 1) [1]
        batch_acc = (buffer == y_train).sum()
        train_crt_pred +=  batch_acc
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
                
        if b%  int((len(train_dataset)/bat_sz)/3 )  == 0:
            print(f'Epoch{i+1:2} Batch {b:4} loss: {loss.item():5.2f} Train Accuracy: {train_crt_pred.item()*100/(bat_sz*b):6.3f}%')
    
    train_loss.append(loss)  #loss after 1 epoch
    train_acc.append(train_crt_pred) # crt predictions after 1 epoch
    
    
    with torch.no_grad():  #testing after 1 complete epoch
        for b,(x_test,y_test) in enumerate (test_loader):
            b += 1
            y_eval = model(x_test)
            loss= criterion(y_eval,y_test.long())
            buffer1 = torch.max(y_eval.data, 1) [1]
            conf_mat =  torch.cat((conf_mat.float(),buffer1.float()),0)
            batch_acc = (buffer1 == y_test).sum()
            test_crt_pred +=  batch_acc
            test_loss.append(loss) #test loss after the last completed epoch
    test_acc.append(test_crt_pred) # crt predictions using the last completed epoch

    print(f'After {i+1} Epoch(s) the Train Accuracy is {(train_crt_pred.item()/len(train_dataset))*100:6.3f}% and Test Accuracy is {(test_crt_pred.item()/len(test_dataset))*100:6.3f}% \n')

In [ ]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns
print('\nThe Confusion Matrix is plotted below:')
cfmt =pd.DataFrame(confusion_matrix(torch.Tensor([r for q,r in test_dataset]).reshape(-1,1),conf_mat.reshape(-1,1),labels=[0,1,2,3,4,5,6]),index=class_names,columns=class_names)
plt.figure(figsize=(12,9))
sns.heatmap(cfmt,annot=True,cmap='BuGn',fmt="d")
plt.xlabel("Prediction",fontsize=14)
plt.ylabel("Labels",fontsize=14)
plt.show()
print('\nThe Classification Report is plotted below: \n\n',classification_report(torch.Tensor([r for q,r in test_dataset]).reshape(-1,1),conf_mat.reshape(-1,1)))

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(train_loss, label='training loss')
plt.plot(test_loss, label='validation loss')
plt.title('Loss at the end of each epoch')
plt.legend();


In [ ]:
plt.figure(figsize=(15,7))
plt.plot([(t/(len(train_dataset)/100)) for t in train_acc], label='Train score',marker='o',alpha=0.8)
plt.plot([(t/(len(test_dataset)/100)) for t in test_acc], label='Test score',marker='o')
plt.title('\nTrain Vs Test Accuracy over Epochs\n',fontsize=16)
# plt.xticks(np.arange(0,20,1));
plt.xlabel('Epochs',fontsize=14)
plt.ylabel('Accuracy Metrics',fontsize=14)
plt.legend();

In [ ]:
# Evaluation DATA TRANSFORMATION
eval_transform =transforms.Compose( [
    transforms.Resize( (48,48)),
    transforms.Grayscale(num_output_channels=1),# Third transformation --Resize to 224 as the mean in the describe function above
    transforms.ToTensor(),                     # Fifth transformation -- to convert it into tensor array (make sure the tensor converion is at the last step)
    
] )

In [ ]:
root = r'C:\Univ'

In [ ]:
import os
from PIL import Image #to operate on pics or images
from IPython.display import display #Only for Jupyter Notebook
import warnings
warnings.filterwarnings('ignore')
eval_data  = datasets.ImageFolder(os.path.join(root,'expr'),transform = eval_transform)

In [ ]:
eval_data

In [ ]:
image, label = eval_data[0]
print('Shape:', image.shape, '\nLabel:', label)

In [ ]:
np.set_printoptions(formatter=dict(int=lambda x: f'{x:5}')) # to widen the printed array
from torchvision.utils import make_grid
# Grab the first  10 images from the first batch of training data
eval_load_all = DataLoader(eval_data, batch_size=2, shuffle=True)
for images,labels in eval_load_all: 
    break

# Print the labels
print('Label:', labels.numpy())
print('Class: ', *np.array([class_names[i] for i in labels]))

# Print the images
im = make_grid(images, nrow=2)  # the default nrow is 8

plt.figure(figsize=(12,4))
plt.imshow(np.transpose(im.numpy(), (1, 2, 0)));

In [ ]:
eval_load_all = DataLoader(eval_data, batch_size=2, shuffle=False)
model.eval()
with torch.no_grad():
    correct = 0
    for X_test, y_test in eval_load_all:
        y_val = model(X_test)
        predicted = torch.max(y_val,1)[1]
        correct += (predicted == y_test).sum()
        
print(f'Test accuracy: {correct.item()}/{len(eval_data)} = {correct.item()*100/(len(eval_data)):7.3f}%')

NameError: ignored